In [1]:
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch import nn

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange

ModuleNotFoundError: ignored

In [0]:
bert_base , vocabulary = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

using cached model
using cached model
using cached model


In [0]:
# transform_text created by BIGTREE

def transform_text(data):
    ds = gluon.data.SimpleDataset([[data]])
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocabulary, lower=False) #vocabulary = gex_mxnet_kobert_model
    trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pair =False)
    res = list(ds.transform(trans))

    input_ids =[]
    input_mask =[]
    token_type_ids=[]

    for data in list(res[0][0]):
        if data ==1 : # which is stopping at padding token (2: CLS, 3:SEP) # 이렇게 쓸 경우 문장의 길이가 모두 달라짐. 옳은 것인가?
            break
        input_ids.append(data)
        input_mask.append(1) # not masking at all
        token_type_ids.append(0) # only one sentence. No NSP.

    input_ids = torch.LongTensor([input_ids])
    input_mask = torch.LongTensor([input_mask])
    token_type_ids = torch.LongTensor([token_type_ids])

    return input_ids, input_mask, token_type_ids 

In [0]:
# Before torch.load, you have to load BERTClassifier.
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 config,
                 num_labels=20,
                 dropout=None,
                 ):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.num_labels = num_labels 
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,num_labels)
       
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

In [0]:
model = torch.load('./mykobert.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
     

In [0]:
def sent2BERT(sent):
    input_ids, input_mask, token_type_ids = transform_text(sent)
    _, pooled_output = model.bert(input_ids)
    vector = np.reshape(pooled_output[0].detach().numpy(),[1,-1])
    
    return vector

In [0]:
df = pd.read_csv("실험 데이터.csv", encoding='cp949')

In [0]:
def load_data(name_data, path):
    os.chdir(path)
    data = pd.read_csv(name_data, encoding='cp949')
    
    rep_data = data['대표 문장']
    rep_data = rep_data.dropna(axis=0)
    
    sim_data = data['유사문장']
    
    tag_data = data['tag']
    return rep_data, sim_data

In [0]:
rep_data, sim_data = load_data("실험 데이터.csv",'./')

In [0]:
rep_vec = [sent2BERT(x) for x in rep_data]
sim_vec = [sent2BERT(x) for x in sim_data]

In [0]:
dfram =pd.DataFrame()

In [0]:
dram[]